In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### Data Reading

In [0]:
df = spark.read.format("parquet")\
       .load("abfss://bronze@databrickproject1.dfs.core.windows.net/products")

df.display()

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df.createOrReplaceTempView("products")
df.display()

### Functions

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price double) 
RETURNS double 
LANGUAGE SQL
return p_price * 0.90;

In [0]:
%sql
select product_id,price, databricks_cata.bronze.discount_func(price) as discounted_price from products

In [0]:
df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_func(price)")) # calling a function in sql way
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand string)
RETURNS STRING 
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$


In [0]:
%sql
SELECT product_id,brand, databricks_cata.bronze.upper_func(brand) as brand_upper  FROM products
    

### Data Writing

In [0]:
df.write.format("delta")\
    .mode('overwrite')\
    .save("abfss://silver@databrickproject1.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products
using delta 
LOCATION "abfss://silver@databrickproject1.dfs.core.windows.net/products"